In [1]:
# --- HCW-enabled version (drop-in) ---
from __future__ import annotations
import math
import time
from dataclasses import dataclass
from typing import Dict, Tuple, Optional

import gymnasium as gym
import numpy as np

try:
    from stable_baselines3 import SAC
    from stable_baselines3.common.monitor import Monitor
    from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize
except Exception:
    SAC = None  # allows import without SB3, e.g., for quick inspection

# ----------------------------
# Environment Implementation
# ----------------------------

@dataclass
class Params:
    dt: float = 0.2
    m: float = 6.0
    I: float = 0.15
    F_max: float = 0.1
    tau_max: float = 0.02
    use_actuator_lag: bool = False
    u_tau: float = 0.2
    u_tau_rot: float = 0.15

    # --- NEW: orbital mean motion [rad/s] for HCW (set directly or compute offline) ---
    # e.g., circular LEO at ~400 km -> n ≈ 0.00113 rad/s (period ~92 min)
    n: float = 0.00113

    # Episode settings
    max_steps: int = 1500
    spawn_r_min: float = 3.0
    spawn_r_max: float = 12.0
    spawn_speed_max: float = 0.05
    spawn_phi_max: float = math.radians(25)
    spawn_omega_max: float = math.radians(1)

    # Success conditions
    dock_radius: float = 0.20
    dock_speed: float = 0.05
    dock_phi: float = math.radians(5)

    # Safety envelope
    soft_bounds: float = 25.0

    # Reward weights
    w_dist: float = 1.0
    w_speed: float = 0.2
    w_phi: float = 0.05
    w_omega: float = 0.01
    w_u: float = 0.005
    w_approach: float = 0.25

    # Terminal rewards
    r_success: float = 200.0
    r_oom: float = -200.0

    # Noise/DR (kept but off by default)
    obs_noise_std: float = 0.0
    domain_rand: bool = False  # keep off so 'n' stays constant
    F_max_range: Tuple[float, float] = (0.08, 0.12)
    tau_max_range: Tuple[float, float] = (0.015, 0.025)
    m_range: Tuple[float, float] = (5.5, 7.0)
    I_range: Tuple[float, float] = (0.12, 0.20)


class RendezvousSE2Env(gym.Env):
    """Underactuated SE2 docking using linear HCW LVLH dynamics in the x–y plane.

    State: [x, y, vx, vy, phi, omega]
      - (x,y) are LVLH radial and along-track relative positions (target at origin)
      - (vx,vy) are LVLH relative velocities
      - phi is chaser body heading wrt LVLH +x (radial-out)
    Action: [a_thrust, a_torque]
      - a_thrust in [-1,1] -> throttle in [0,1] (forward-only)
      - a_torque in [-1,1] -> torque in [-tau_max, +tau_max]
    """
    metadata = {"render_modes": ["human"], "render_fps": 10}

    def __init__(self, params: Optional[Params] = None, render_mode: Optional[str] = None):
        super().__init__()
        self.p = params or Params()
        self.render_mode = render_mode

        self.state = np.zeros(6, dtype=np.float32)
        self._F = 0.0
        self._tau = 0.0

        # Fixed physical params (you can re-enable DR if you like)
        self._m = self.p.m
        self._I = self.p.I
        self._F_max = self.p.F_max
        self._tau_max = self.p.tau_max

        high_obs = np.array([
            self.p.soft_bounds, self.p.soft_bounds,
            3.0, 3.0,
            math.pi, 2.0
        ], dtype=np.float32)
        self.observation_space = gym.spaces.Box(low=-high_obs, high=high_obs, dtype=np.float32)
        self.action_space = gym.spaces.Box(low=np.array([-1.0, -1.0], dtype=np.float32),
                                           high=np.array([1.0, 1.0], dtype=np.float32), dtype=np.float32)
        self._step_count = 0

    # --- helpers (unchanged) ---
    def _success(self, x, y, vx, vy, phi, omega) -> bool:
        dist = math.hypot(x, y)
        speed = math.hypot(vx, vy)
        return (dist <= self.p.dock_radius) and (speed <= self.p.dock_speed) and (abs(phi) <= self.p.dock_phi)

    def _out_of_bounds(self, x, y) -> bool:
        return (abs(x) > self.p.soft_bounds) or (abs(y) > self.p.soft_bounds)

    def reset(self, *, seed: Optional[int] = None, options: Optional[Dict] = None):
        super().reset(seed=seed)
        self._step_count = 0
        rng = np.random.default_rng(seed)

        # Spawn around ring in LVLH plane
        r = float(rng.uniform(self.p.spawn_r_min, self.p.spawn_r_max))
        theta = float(rng.uniform(-math.pi, math.pi))
        x, y = r * math.cos(theta), r * math.sin(theta)
        vx = float(rng.uniform(-self.p.spawn_speed_max, self.p.spawn_speed_max))
        vy = float(rng.uniform(-self.p.spawn_speed_max, self.p.spawn_speed_max))
        phi = float(rng.uniform(-self.p.spawn_phi_max, self.p.spawn_phi_max))
        omega = float(rng.uniform(-self.p.spawn_omega_max, self.p.spawn_omega_max))

        self.state = np.array([x, y, vx, vy, phi, omega], dtype=np.float32)
        self._F, self._tau = 0.0, 0.0
        return self._observe(), {}

    def _observe(self):
        obs = self.state.copy()
        if self.p.obs_noise_std > 0.0:
            obs += np.random.normal(0.0, self.p.obs_noise_std, size=obs.shape).astype(np.float32)
        np.clip(obs, self.observation_space.low, self.observation_space.high, out=obs)
        return obs

    def step(self, action: np.ndarray):
        self._step_count += 1
        a_thrust = float(np.clip(action[0], -1.0, 1.0))
        a_torque = float(np.clip(action[1], -1.0, 1.0))

        # Map to physical commands
        throttle = 0.5 * (a_thrust + 1.0)  # [-1,1] -> [0,1]
        F_cmd = throttle * self._F_max
        tau_cmd = a_torque * self._tau_max

        # Actuator lag (optional)
        if self.p.use_actuator_lag:
            self._F += (self.p.dt / self.p.u_tau) * (F_cmd - self._F)
            self._tau += (self.p.dt / self.p.u_tau_rot) * (tau_cmd - self._tau)
        else:
            self._F, self._tau = F_cmd, tau_cmd

        # Unpack
        x, y, vx, vy, phi, omega = map(float, self.state)
        dt = self.p.dt
        inv_m = 1.0 / self._m
        inv_I = 1.0 / self._I
        n = self.p.n

        # Body-forward thruster -> LVLH components
        a_tx = (self._F * inv_m) * math.cos(phi)
        a_ty = (self._F * inv_m) * math.sin(phi)
        alpha = self._tau * inv_I

        # --- HCW translational dynamics in LVLH (x: radial, y: along-track) ---
        # xdd - 3 n^2 x - 2 n ydot = a_tx
        # ydd + 2 n xdot = a_ty
        xdd = 3.0 * (n ** 2) * x + 2.0 * n * vy + a_tx
        ydd = -2.0 * n * vx + a_ty

        # Semi-implicit Euler integration
        vx += xdd * dt
        vy += ydd * dt
        x  += vx  * dt
        y  += vy  * dt

        omega += alpha * dt
        phi   += omega * dt
        phi = (phi + math.pi) % (2 * math.pi) - math.pi  # wrap

        # Write back
        s = self.state
        s[0] = x; s[1] = y; s[2] = vx; s[3] = vy; s[4] = phi; s[5] = omega
        self.state = s

        # --- Rewards & termination (unchanged) ---
        dist = math.hypot(x, y)
        speed = math.hypot(vx, vy)
        approach_speed = -((x * vx + y * vy) / dist) if dist > 1e-6 else 0.0

        r = 0.0
        r -= self.p.w_dist * dist
        r -= self.p.w_speed * speed
        r -= self.p.w_phi * abs(phi)
        r -= self.p.w_omega * abs(omega)
        r += self.p.w_approach * approach_speed
        r -= self.p.w_u * ((abs(self._F) / (self._F_max + 1e-9)) + (abs(self._tau) / (self._tau_max + 1e-9)))

        terminated = False
        truncated = False
        info: Dict = {}

        if self._success(x, y, vx, vy, phi, omega):
            r += self.p.r_success
            terminated = True
            info["is_success"] = True

        if self._out_of_bounds(x, y):
            r += self.p.r_oom
            terminated = True
            info["failure"] = "out_of_bounds"

        if self._step_count >= self.p.max_steps:
            truncated = True

        return self._observe(), float(r), terminated, truncated, info

    def render(self):
        x, y, vx, vy, phi, omega = self.state
        print(f"t={self._step_count*self.p.dt:6.2f}s  pos=({x:6.2f},{y:6.2f})  vel=({vx:5.2f},{vy:5.2f})  phi={math.degrees(phi):6.2f}°  w={math.degrees(omega):6.2f}°/s")



In [2]:

# ----------------------------
# Training Pipeline (SAC)
# ----------------------------

def make_env(params: Optional[Params] = None):
    def _thunk():
        env = RendezvousSE2Env(params=params)
        env = Monitor(env)
        return env
    return _thunk


def train_sac(total_timesteps: int = 300_000, n_envs: int = 8, seed: int = 42,
              normalize: bool = True, params: Optional[Params] = None,
              log_dir: str = "./logs_se2"):
    if SAC is None:
        raise RuntimeError("stable-baselines3 not installed. Run: pip install stable-baselines3")

    vec_env = DummyVecEnv([make_env(params) for _ in range(n_envs)])

    if normalize:
        vec_env = VecNormalize(vec_env, norm_obs=True, norm_reward=True, clip_obs=10.0, gamma=0.995)

    model = SAC(
        policy="MlpPolicy",
        env=vec_env,
        learning_rate=3e-4,
        buffer_size=1_000_000,
        batch_size=256,
        tau=0.02,
        gamma=0.995,
        train_freq=1,
        gradient_steps=1,
        policy_kwargs=dict(net_arch=[256, 256]),
        verbose=1,
        seed=seed,
    )

    model.learn(total_timesteps=total_timesteps, progress_bar=True)

    # Save model and normalization (if used)
    ts = int(time.time())
    model.save(f"sac_se2_{ts}")
    if normalize and isinstance(vec_env, VecNormalize):
        vec_env.save(f"vecnorm_se2_{ts}.pkl")

    try:
        vec_env.close()
    except Exception:
        pass




# ----------------------------
# Rollout + Plotting Utilities
# ----------------------------
import matplotlib.pyplot as plt


def rollout(env: RendezvousSE2Env, policy=None, max_steps: int = None, seed: int = 0):
    """Run one episode and collect history.
    - policy: callable(obs)->action in env.action_space; if None, sample random.
    Returns a dict with arrays.
    """
    obs, info = env.reset(seed=seed)
    done = False
    terminated = False
    truncated = False
    steps = 0
    max_steps = max_steps or env.p.max_steps

    hist = {
        "t": [],
        "x": [], "y": [], "vx": [], "vy": [], "phi": [], "omega": [],
        "F": [], "tau": [], "a0": [], "a1": [], "r": [], "dist": [], "speed": [],
    }

    while not (terminated or truncated) and steps < max_steps:
        if policy is None:
            action = env.action_space.sample()
        else:
            action = np.asarray(policy(obs), dtype=np.float32)

        # Step
        obs, reward, terminated, truncated, info = env.step(action)

        # Unpack state for logging
        x, y, vx, vy, phi, omega = env.state
        F = env._F
        tau = env._tau
        dist = float(np.hypot(x, y))
        speed = float(np.hypot(vx, vy))

        t = steps * env.p.dt
        hist["t"].append(t)
        hist["x"].append(float(x)); hist["y"].append(float(y))
        hist["vx"].append(float(vx)); hist["vy"].append(float(vy))
        hist["phi"].append(float(phi)); hist["omega"].append(float(omega))
        hist["F"].append(float(F)); hist["tau"].append(float(tau))
        hist["a0"].append(float(action[0])); hist["a1"].append(float(action[1]))
        hist["r"].append(float(reward))
        hist["dist"].append(dist); hist["speed"].append(speed)

        steps += 1

    for k in hist:
        hist[k] = np.asarray(hist[k])

    hist["is_success"] = bool(info.get("is_success", False))
    hist["failure"] = info.get("failure")
    # Include key environment parameters used for plotting to avoid relying on globals
    try:
        hist["dock_radius"] = float(env.p.dock_radius)
        hist["dock_speed"] = float(env.p.dock_speed)
        hist["dock_phi"] = float(env.p.dock_phi)
        hist["soft_bounds"] = float(env.p.soft_bounds)
    except Exception:
        # Fallbacks if env does not have expected attributes
        hist["dock_radius"] = hist.get("dock_radius", 0.20)
        hist["dock_speed"] = hist.get("dock_speed", 0.05)
        hist["dock_phi"] = hist.get("dock_phi", math.radians(5))
        hist["soft_bounds"] = hist.get("soft_bounds", 25.0)
    return hist


def make_policy_from_sb3(model):
    """Wrap a Stable-Baselines3 model into a callable(obs)->action suitable for rollout."""
    def _pi(obs):
        # SB3 expects batch; deterministic=True for evaluation
        action, _ = model.predict(obs, deterministic=True)
        return action
    return _pi


def plot_rollout(hist: dict, title: str = "SE2 Rendezvous Rollout", show: bool = True, save_path: str = None):
    """Plot XY trajectory, time series of distance/speed, attitude rates, and controls."""
    t = hist["t"]
    x, y = hist["x"], hist["y"]
    vx, vy = hist["vx"], hist["vy"]
    phi, omega = hist["phi"], hist["omega"]
    F, tau = hist["F"], hist["tau"]
    dist, speed = hist["dist"], hist["speed"]

    # Pull plotting params from hist (recorded during rollout)
    dock_radius = float(hist.get("dock_radius", 0.20))
    dock_speed = float(hist.get("dock_speed", 0.05))
    dock_phi = float(hist.get("dock_phi", math.radians(5)))
    soft_bounds = float(hist.get("soft_bounds", 25.0))

    # Figure 1: XY trajectory with headings (quiver every N)
    plt.figure(figsize=(6, 6))
    plt.plot(x, y, linewidth=2)
    # Docking zone & bounds
    ax = plt.gca()
    dock = plt.Circle((0, 0), radius=dock_radius, fill=False, linestyle='--')
    ax.add_artist(dock)
    sb = soft_bounds
    ax.set_xlim(-sb, sb); ax.set_ylim(-sb, sb)
    ax.set_aspect('equal', adjustable='box')
    plt.title(f"{title} — XY path (success={hist['is_success']})")
    plt.xlabel("x [m]"); plt.ylabel("y [m]")
    # Heading arrows
    N = max(1, len(x)//25)
    u = np.cos(phi)[::N]
    v = np.sin(phi)[::N]
    plt.quiver(x[::N], y[::N], u, v, angles='xy', scale_units='xy', scale=5)
    if save_path:
        plt.savefig(f"{save_path}_xy.png", dpi=150, bbox_inches='tight')

    # Figure 2: Distance & speed
    plt.figure(figsize=(8, 4))
    plt.plot(t, dist, label='distance [m]')
    plt.plot(t, speed, label='speed [m/s]')
    plt.axhline(dock_radius, linestyle='--', linewidth=1)
    plt.axhline(dock_speed, linestyle='--', linewidth=1)
    plt.xlabel('time [s]'); plt.ylabel('metric'); plt.legend(); plt.title('Range & Speed vs. Time')
    if save_path:
        plt.savefig(f"{save_path}_range_speed.png", dpi=150, bbox_inches='tight')

    # Figure 3: Attitude & rate
    plt.figure(figsize=(8, 4))
    plt.plot(t, np.degrees(phi), label='phi [deg]')
    plt.plot(t, np.degrees(omega), label='omega [deg/s]')
    plt.axhline(np.degrees(dock_phi), linestyle='--', linewidth=1)
    plt.axhline(-np.degrees(dock_phi), linestyle='--', linewidth=1)
    plt.xlabel('time [s]'); plt.ylabel('attitude'); plt.legend(); plt.title('Attitude vs. Time')
    if save_path:
        plt.savefig(f"{save_path}_attitude.png", dpi=150, bbox_inches='tight')

    # Figure 4: Controls & reward
    plt.figure(figsize=(8, 4))
    plt.plot(t, F, label='F [N]')
    plt.plot(t, tau, label='tau [N·m]')
    plt.xlabel('time [s]'); plt.ylabel('actuators'); plt.legend(); plt.title('Actuators vs. Time')
    if save_path:
        plt.savefig(f"{save_path}_actuators.png", dpi=150, bbox_inches='tight')

    if show:
        plt.show()
    else:
        plt.close('all')


# if __name__ == "__main__":
#     # Example: evaluate a random policy (replace with trained SB3 model load)
#     env = RendezvousSE2Env()

#     # To use a trained model, uncomment and set the path:
#     from stable_baselines3 import SAC
#     model = SAC.load("sac_se2_1756564950")
#     policy = make_policy_from_sb3(model)
#     hist = rollout(env, policy=policy, seed=123)

#     hist = rollout(env, policy=None, seed=123)
#     plot_rollout(hist, title="Debug Rollout", show=True, save_path=None)


# # ----------------------------
# # Quick smoke test & training
# # ----------------------------
# if __name__ == "__main__":
#     # Quick environment sanity check
#     env = RendezvousSE2Env()
#     obs, info = env.reset(seed=0)
#     for _ in range(5):
#         a = env.action_space.sample()
#         obs, r, term, trunc, info = env.step(a)
#         env.render()
#         if term or trunc:
#             obs, info = env.reset()

#     # Kick off a short train by default (adjust timesteps for serious runs)
#     if SAC is not None:
#         p = Params()
#         train_sac(total_timesteps=100_000, n_envs=4, params=p)
#     else:
#         print("Stable-Baselines3 not found; skipping training. Install it to train the agent.")









"""
Enhanced Training Loop with Periodic Evaluation & Plotting
---------------------------------------------------------
- Extends the quick training main to:
  * periodically evaluate the SAC agent on a single environment
  * generate rollout plots every N steps
  * log intermediate results
"""

import os
import matplotlib.pyplot as plt


def evaluate_and_plot(model, env, step, outdir="evals"):
    os.makedirs(outdir, exist_ok=True)
    policy = make_policy_from_sb3(model)
    hist = rollout(env, policy=policy, seed=step)
    save_path = os.path.join(outdir, f"eval_{step}")
    plot_rollout(hist, title=f"Eval at step {step}", show=False, save_path=save_path)
    print(f"[Eval] step={step}, success={hist['is_success']}, saved plots to {save_path}_*.png")


if __name__ == "__main__":
    """Enhanced training main with periodic evaluation plots.
    - Trains SAC with VecNormalize
    - Every `plot_every_steps`, runs a few eval episodes and saves plots under ./plots_sac/<ts>/
    """
    import os
    from stable_baselines3.common.callbacks import BaseCallback

    class TrainPlotCallback(BaseCallback):
        def __init__(self, params: Params, plot_every_steps: int = 50_000, eval_episodes: int = 2, out_dir: str = "./plots_sac", verbose: int = 0):
            super().__init__(verbose)
            self.params = params
            self.plot_every_steps = int(plot_every_steps)
            self.eval_episodes = int(eval_episodes)
            self.out_dir = out_dir
            self.ts = str(int(time.time()))
            os.makedirs(os.path.join(out_dir, self.ts), exist_ok=True)

        def _init_callback(self) -> None:
            # nothing special
            return None

        def _on_step(self) -> bool:
            if self.num_timesteps % self.plot_every_steps != 0:
                return True
            # Build a fresh eval env
            base_env = RendezvousSE2Env(self.params)
            eval_env = DummyVecEnv([lambda: Monitor(base_env)])
            # If training uses VecNormalize, mirror obs stats for proper eval
            if isinstance(self.model.get_env(), VecNormalize):
                eval_env = VecNormalize(eval_env, training=False, norm_obs=False, norm_reward=False)
                # eval_env.obs_rms = self.model.get_env().obs_rms

            #also save everything related to the actor, environment and so on
            self.model.save(os.path.join(self.out_dir, self.ts, f"model_{self.num_timesteps}"))

            # Roll a few eval episodes and plot
            pi = make_policy_from_sb3(self.model)
            for ep in range(self.eval_episodes):
                hist = rollout(base_env, policy=pi, seed=ep)
                tag = f"step_{self.num_timesteps}_ep_{ep}"
                save_path = os.path.join(self.out_dir, self.ts, tag)
                plot_rollout(hist, title=f"SAC eval @ {self.num_timesteps}", show=False, save_path=save_path)
            if self.verbose:
                print(f"[PlotCallback] Saved eval plots at {self.num_timesteps} steps.")
            return True

    # -------- Quick environment sanity check (optional) --------
    env_debug = RendezvousSE2Env()
    obs, info = env_debug.reset(seed=0)
    for _ in range(3):
        a = env_debug.action_space.sample()
        obs, r, term, trunc, info = env_debug.step(a)
        if term or trunc:
            obs, info = env_debug.reset()

    # -------- Training with periodic plots --------
    if SAC is not None:
        p = Params()
        n_envs = 4
        total_timesteps = 60_000_000

        # Vectorized env with normalization
        vec = DummyVecEnv([make_env(p) for _ in range(n_envs)])
        vec = VecNormalize(vec, norm_obs=False, norm_reward=True, clip_obs=10.0, gamma=0.995)

        model = SAC(
            policy="MlpPolicy",
            env=vec,
            learning_rate=3e-4,
            buffer_size=1_000_000,
            batch_size=256,
            tau=0.05,
            gamma=0.99,
            train_freq=(50, "step"),
            gradient_steps=-1,
            policy_kwargs=dict(net_arch=[256, 256, 64]),
            verbose=1,
            seed=42,
            tensorboard_log="./tb_logs_se2",
        )

        cb = TrainPlotCallback(params=p, plot_every_steps=20_000, eval_episodes=3, out_dir="./plots_sac", verbose=1)
        model.learn(total_timesteps=total_timesteps, progress_bar=True, callback=cb)

        # Save artifacts
        ts = int(time.time())
        model.save(f"sac_se2_{ts}")
        if isinstance(vec, VecNormalize):
            vec.save(f"vecnorm_se2_{ts}.pkl")
    else:
        print("Stable-Baselines3 not found; skipping training. Install it to train the agent.")


Using cuda device
Logging to ./tb_logs_se2/SAC_12


/home/simone/.miniforge3/envs/SafeRL-312/lib/python3.12/site-packages/rich/live.py:256: UserWarning: install 
"ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

----------------------------------
| rollout/           |           |
|    ep_len_mean     | 1.04e+03  |
|    ep_rew_mean     | -1.37e+04 |
| time/              |           |
|    episodes        | 4         |
|    fps             | 167       |
|    time_elapsed    | 35        |
|    total_timesteps | 6000      |
| train/             |           |
|    actor_loss      | -35.4     |
|    critic_loss     | 0.571     |
|    ent_coef        | 0.182     |
|    ent_coef_loss   | -5.56     |
|    learning_rate   | 0.0003    |
|    n_updates       | 5800      |
----------------------------------
----------------------------------
| rollout/           |           |
|    ep_len_mean     | 1.03e+03  |
|    ep_rew_mean     | -1.34e+04 |
| time/              |           |
|    episodes        | 8         |
|    fps             | 164       |
|    time_elapsed    | 60        |
|    total_timesteps | 10012     |
| train/             |           |
|    actor_loss      | -7.96     |
|    critic_loss    

[PlotCallback] Saved eval plots at 20000 steps.

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1.03e+03 |
|    ep_rew_mean     | -1.3e+04 |
| time/              |          |
|    episodes        | 20       |
|    fps             | 164      |
|    time_elapsed    | 130      |
|    total_timesteps | 21512    |
| train/             |          |
|    actor_loss      | 1.2      |
|    critic_loss     | 0.00267  |
|    ent_coef        | 0.00178  |
|    ent_coef_loss   | -16.4    |
|    learning_rate   | 0.0003   |
|    n_updates       | 21400    |
---------------------------------
----------------------------------
| rollout/           |           |
|    ep_len_mean     | 965       |
|    ep_rew_mean     | -1.24e+04 |
| time/              |           |
|    episodes        | 24        |
|    fps             | 164       |
|    time_elapsed    | 154       |
|    total_timesteps | 25512     |
| train/             |           |
|    actor_loss      | 1.23      |
|    critic_loss     | 0.00236   |
| 

...

[PlotCallback] Saved eval plots at 4420000 steps.

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 239      |
|    ep_rew_mean     | -809     |
|    success_rate    | 1        |
| time/              |          |
|    episodes        | 15540    |
|    fps             | 163      |
|    time_elapsed    | 26960    |
|    total_timesteps | 4420228  |
| train/             |          |
|    actor_loss      | 0.418    |
|    critic_loss     | 3.71e-05 |
|    ent_coef        | 0.000134 |
|    ent_coef_loss   | -0.338   |
|    learning_rate   | 0.0003   |
|    n_updates       | 4420200  |
---------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 237      |
|    ep_rew_mean     | -805     |
|    success_rate    | 1        |
| time/              |          |
|    episodes        | 15544    |
|    fps             | 163      |
|    time_elapsed    | 26965    |
|    total_timesteps | 4421116  |
| train/             |          |
|    actor_los

[PlotCallback] Saved eval plots at 4440000 steps.

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 240      |
|    ep_rew_mean     | -843     |
|    success_rate    | 1        |
| time/              |          |
|    episodes        | 15624    |
|    fps             | 163      |
|    time_elapsed    | 27080    |
|    total_timesteps | 4440356  |
| train/             |          |
|    actor_loss      | 0.424    |
|    critic_loss     | 3.05e-05 |
|    ent_coef        | 0.00013  |
|    ent_coef_loss   | 0.197    |
|    learning_rate   | 0.0003   |
|    n_updates       | 4440200  |
---------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 242      |
|    ep_rew_mean     | -855     |
|    success_rate    | 1        |
| time/              |          |
|    episodes        | 15628    |
|    fps             | 163      |
|    time_elapsed    | 27086    |
|    total_timesteps | 4441344  |
| train/             |          |
|    actor_los

KeyboardInterrupt: 

In [ ]:
# ------------------------------------------------------
# Animated GIF of the rendezvous rollout (XY view)
# ------------------------------------------------------
# Requires: matplotlib (and Pillow for GIF saving)
#   pip install matplotlib pillow
#
# Usage after training:
#   env = RendezvousSE2Env()
#   model = SAC.load("sac_se2_....zip")
#   policy = make_policy_from_sb3(model)
#   hist = rollout(env, policy=policy, seed=0)
#   animate_rollout(hist, out_path="rendezvous.gif", fps=20)
#
from typing import Optional
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation
from matplotlib.patches import Polygon, Circle, FancyArrowPatch

def animate_rollout(
    hist: dict,
    out_path: str = "rendezvous.gif",
    fps: int = 20,
    dpi: int = 150,
    trail_seconds: float = 20.0,     # how long the trail should be (seconds)
    body_len: float = 0.6,           # chaser body length (plot units, meters)
    body_wid: float = 0.35,          # chaser body width  (plot units, meters)
    show_velocity: bool = False,
    show_thruster: bool = True,
    show_bounds: bool = True,
    facecolor: str = "white",
):
    """
    Create an animated GIF from a rollout history dict.

    hist keys expected (from your rollout): 
      t, x, y, phi, F, dist, speed, (optional) dock_radius, soft_bounds
    """
    # Unpack series
    t = np.asarray(hist["t"])
    x = np.asarray(hist["x"])
    y = np.asarray(hist["y"])
    phi = np.asarray(hist["phi"])

    # Optionals / defaults
    dist = np.asarray(hist.get("dist", np.hypot(x, y)))
    speed = np.asarray(hist.get("speed", np.zeros_like(t)))
    F = np.asarray(hist.get("F", np.zeros_like(t)))
    dock_radius = float(hist.get("dock_radius", 0.20))
    soft_bounds = float(hist.get("soft_bounds", 25.0))

    # Timeline / frames
    n = len(t)
    if n == 0:
        raise ValueError("Empty rollout history.")
    dt = float(t[1] - t[0]) if len(t) > 1 else 0.2
    trail_frames = max(1, int(trail_seconds / max(dt, 1e-9)))

    # Figure & axes
    fig, ax = plt.subplots(figsize=(6, 6), facecolor=facecolor)
    ax.set_aspect("equal", adjustable="box")
    ax.set_xlim(-soft_bounds, soft_bounds)
    ax.set_ylim(-soft_bounds, soft_bounds)
    ax.set_xlabel("x [m]")
    ax.set_ylabel("y [m]")
    title_txt = ax.set_title("Rendezvous animation")

    # Target + docking zone
    dock = Circle((0, 0), radius=dock_radius, fill=False, linestyle="--", linewidth=1)
    ax.add_patch(dock)

    # Static crosshair for the target at origin
    ax.axhline(0, color="0.85", linewidth=0.8)
    ax.axvline(0, color="0.85", linewidth=0.8)

    # Optional soft bounds rectangle
    if show_bounds:
        ax.plot([ soft_bounds, soft_bounds,-soft_bounds,-soft_bounds, soft_bounds],
                [ soft_bounds,-soft_bounds,-soft_bounds, soft_bounds, soft_bounds],
                linestyle=":", color="0.8", linewidth=1)

    # Full path (light), updated trail (bold)
    path_all, = ax.plot(x, y, color="0.85", linewidth=1, zorder=1)
    trail,    = ax.plot([], [], color="tab:blue", linewidth=2.0, zorder=2)

    # Chaser body (triangle) in local coords, then we’ll rotate/translate each frame
    # Triangle pointing +x_body: nose, rear-left, rear-right
    tri_local = np.array([
        [ +body_len/2,  0.0            ],
        [ -body_len/2, +body_wid/2     ],
        [ -body_len/2, -body_wid/2     ],
    ])
    body_patch = Polygon(tri_local, closed=True, color="tab:blue", alpha=0.85, zorder=3)
    ax.add_patch(body_patch)

    # Velocity arrow (world frame)
    vel_arrow: Optional[ FancyArrowPatch ] = None
    if show_velocity:
        vel_arrow = FancyArrowPatch(
            posA=(0,0), posB=(0,0),
            arrowstyle="-|>", mutation_scale=10,
            linewidth=1.0, color="tab:green", zorder=3
        )
        ax.add_patch(vel_arrow)

    # Thruster plume: draw a short line behind the body along -x_body
    thr_line, = ax.plot([], [], linewidth=2.5, color="tab:orange", alpha=0.9, zorder=2)
    # Normalize F for display
    F_scale = (np.max(np.abs(F)) + 1e-9)

    # Text status box
    info = ax.text(
        0.02, 0.98, "", transform=ax.transAxes,
        ha="left", va="top", fontsize=9,
        bbox=dict(boxstyle="round,pad=0.2", facecolor="white", alpha=0.8)
    )

    # Helpers
    def rot2d(theta):
        c, s = np.cos(theta), np.sin(theta)
        return np.array([[c, -s], [s, c]])

    def body_world_vertices(cx, cy, heading):
        R = rot2d(heading)
        return tri_local @ R.T + np.array([cx, cy])

    def update_frame(i):
        # Trail window
        j0 = max(0, i - trail_frames)
        trail.set_data(x[j0:i+1], y[j0:i+1])

        # Body pose
        verts = body_world_vertices(x[i], y[i], phi[i])
        body_patch.set_xy(verts)

        # Velocity arrow
        if show_velocity and vel_arrow is not None:
            # Differentiate position if speed not available
            if i > 0:
                vx = (x[i] - x[i-1]) / max(dt, 1e-9)
                vy = (y[i] - y[i-1]) / max(dt, 1e-9)
            else:
                vx = vy = 0.0
            vlen = np.hypot(vx, vy) + 1e-12
            scale = body_len * 1.25 / vlen if vlen > 0 else 0.0
            vdx, vdy = vx * scale, vy * scale
            vel_arrow.set_positions((x[i], y[i]), (x[i] + vdx, y[i] + vdy))

        # Thruster plume (behind body along -x_body); length ~ thrust
        if show_thruster:
            # In body frame, start slightly behind rear; end farther w.r.t. throttle
            # Unit -x_body in world:
            ux = -np.cos(phi[i])
            uy = -np.sin(phi[i])
            # Base point ~ center of rear edge
            rear_center = (verts[1] + verts[2]) * 0.5
            plume_len = (abs(F[i]) / F_scale) * (body_len * 0.9)
            xs = [rear_center[0], rear_center[0] + plume_len * ux]
            ys = [rear_center[1], rear_center[1] + plume_len * uy]
            thr_line.set_data(xs, ys)
            thr_line.set_alpha(0.35 + 0.65 * (abs(F[i]) / F_scale))
        else:
            thr_line.set_data([], [])

        # Title & HUD text
        title_txt.set_text(f"Rendezvous (frame {i+1}/{n})")
        info.set_text(
            f"t = {t[i]:.1f} s\n"
            f"r = {dist[i]:.2f} m  |  v = {speed[i]:.2f} m/s\n"
            f"phi = {np.degrees(phi[i]):.1f}°"
        )

        return (trail, body_patch, thr_line, title_txt, info) if vel_arrow is None else \
               (trail, body_patch, thr_line, vel_arrow, title_txt, info)

    interval_ms = int(1000 / fps)
    ani = animation.FuncAnimation(
        fig, update_frame, frames=n, interval=interval_ms, blit=False
    )

    # Save GIF (requires Pillow)
    print(f"[anim] Saving GIF to {out_path} (fps={fps}, dpi={dpi}) ...")
    ani.save(out_path, writer="pillow", fps=fps, dpi=dpi)
    plt.close(fig)
    print("[anim] Done!")



In [5]:
# after you’ve trained your SAC model…
env = RendezvousSE2Env()
from stable_baselines3 import SAC
model = SAC.load("plots_sac/1756814474/model_4440000.zip")
policy = make_policy_from_sb3(model)

# run one eval episode and animate it
hist = rollout(env, policy=policy, seed=0)
animate_rollout(hist, out_path="rendezvous.gif", fps=20, trail_seconds=25.0)


[anim] Saving GIF to rendezvous.gif (fps=20, dpi=150) ...

/home/simone/.miniforge3/envs/SafeRL-312/lib/python3.12/site-packages/rich/live.py:256: UserWarning: install 
"ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

[anim] Done!